# Part 2: Viewshed case study

In the second part, we will demonstrate the use of GRASS for a small viewshed case study.
The goal is to **compute and analyze the area a driver would see from a road**.
This notebook can be run only after notebook Part 1 was executed.

Topics covered:
 * Python scripting
 * manipulating vector data ([v.build.polylines](https://grass.osgeo.org/grass-stable/manuals/v.build.polylines.html), [v.to.points](https://grass.osgeo.org/grass-stable/manuals/v.to.points.html))
 * vector attributes ([v.db.select](https://grass.osgeo.org/grass-stable/manuals/v.db.select.html))
 * viewshed computation ([r.viewshed](https://grass.osgeo.org/grass-stable/manuals/r.viewshed.html))
 * simple parallelization ([multiprocessing.Pool](https://docs.python.org/3/library/multiprocessing.html))
 * region handling ([grass.script.region_env](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#script.core.region_env))
 * raster algebra ([r.mapcalc](https://grass.osgeo.org/grass-stable/manuals/r.mapcalc.html))
 * temporal data handling
 * reprojecting ([r.proj](https://grass.osgeo.org/grass-stable/manuals/r.proj.html))
 * raster mask ([r.mask](https://grass.osgeo.org/grass-stable/manuals/r.mask.html))
 * raster as numpy array ([grass.script.array](https://grass.osgeo.org/grass-stable/manuals/libpython/script.html#module-script.array))

In the previous notebook (Part 1) we created new location (project) *dix_park*. This automatically created new mapset (subproject) _PERMANENT_ where we then imported our base data. Now it's time to create a new mapset for our viewshed analysis, we will name it _viewshed_:

In [ ]:
%%bash
grass -c -e ~/grassdata/dix_park/viewshed

In [ ]:
# Import Python standard library and IPython packages we need.
import subprocess
import sys
from tqdm import tqdm

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "dix_park", "viewshed")

## Data preparation
We will first derive viewpoints along the road *Umstead Drive* (vector `umstead_drive_segments`) that we extracted in the first part of the workshop.

1. Because the road consists of several segments, we will merge them into one.
2. Create new vector of points along the line with distance 50 m.

In [ ]:
gs.run_command("v.build.polylines", input="umstead_drive_segments", output="umstead_drive", cats="first")
gs.run_command("v.to.points", input="umstead_drive", type="line", output="viewpoints", dmax=50)

Visualize the points with InteractiveMap with OSM tiles (see [other tile options](https://grass.osgeo.org/grass-stable/manuals/libpython/grass.jupyter.html#module-grass.jupyter.interactivemap)):

In [ ]:
road_map = gj.InteractiveMap(tiles="OpenStreetMap")
road_map.add_vector("umstead_drive")
road_map.add_vector("viewpoints")
road_map.show()

Next part of analysis is raster-based, so we need to make sure we set computational region as we need. Specifically, we set it to match the DSM:

In [ ]:
gs.run_command("g.region", raster="dsm")

Now we want to compute the visibility using DSM, however some points may fall on top of a tree, so we need to filter those out.

First compute height above ground (DSM - DTM):

In [ ]:
gs.mapcalc("diff = dsm - ground")
gs.run_command("r.colors", map="diff", color="differences")

diff_map = gj.Map()
diff_map.d_rast(map="diff")
diff_map.d_vect(map="umstead_drive")
diff_map.d_legend(raster="diff")
diff_map.show()

Extract height above ground for the viewpoint locations to identify points that fall on top of a tree growing next to the road:

In [ ]:
gs.run_command("v.what.rast", map="viewpoints", layer=2, raster="diff", column="height")

See the newly computed attribute data. This example shows how the attribute data can be loaded into pandas:

In [ ]:
import json
import pandas as pd
pd.DataFrame(json.loads(gs.read_command("v.db.select", map="viewpoints", columns="cat,height", layer=2, format="json"))["records"])

Visualize the viewpoints with the height-above-ground raster. You can filter the points based on the height above ground, we won't display points with height > 2.
Additionally, we will render the result larger (`width=1000`) and we will render the map zoomed in to the area with the points
by saving a region and using it in Map (`saved_region="umstead_drive_region"`).

In [ ]:
gs.run_command("g.region", vector="umstead_drive", align="dsm", save="umstead_drive_region")

img = gj.Map(width=1000, saved_region="umstead_drive_region")
img.d_rast(map="diff")
img.d_vect(map="umstead_drive")
img.d_vect(map="viewpoints", layer=2, where="height >= 2", size=15, icon="basic/pin", fill_color="red")
img.d_vect(map="viewpoints", layer=2, where="height < 2", size=15, icon="basic/pin")
img.d_legend(raster="diff")
img.show()

## Viewshed computation
To get the cumulative viewshed, we will compute viewsheds from all the viewpoints we generated earlier.
First, we get the list coordinates of the viewpoints that are likely lying on the ground:

In [ ]:
viewpoints = gs.read_command('v.out.ascii', input='viewpoints',
                             separator='comma', layer=2, where="height < 2").strip().splitlines()
viewpoints = [p.split(",") for p in viewpoints]
viewpoints

We will now compute the viewshed from each viewpoint in a loop. We set max distance of 300 m. Each viewshed will be named `viewshed_cat`.

In [ ]:
%%time
maps = []
for x, y, cat in tqdm(viewpoints):
    name = f"viewshed_{cat}"
    gs.run_command("r.viewshed", input="dsm", output=name,
                   coordinates=(x, y), max_distance=300, flags="b")
    maps.append(name)

Since these are independent runs, we can easily parallelize the r.viewshed calls using Python multiprocessing.
We define a function that computes the viewshed and returns the name of the output or None in case of error:

In [ ]:
%%time
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name,
                       coordinates=(x, y), max_distance=300, flags="b")
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)

One trick to speedup viewshed computation is to limit the computation only
to the actual area given by the maxdistance option. To do that we will locally modify the computational region
and pass the environment to the module directly. The current computational region won't be affected.

In [ ]:
%%time
import os
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # copy current environment
    env = os.environ.copy()
    # set GRASS_REGION variable using region_env function
    env["GRASS_REGION"] = gs.region_env(align="dsm",
                                        e=x + max_distance,
                                        w=x - max_distance,
                                        n=y + max_distance,
                                        s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name, flags="b",
                      coordinates=(x, y), max_distance=max_distance, env=env)
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 1
with Pool(processes=proc) as pool:
    maps = pool.map_async(viewshed, viewpoints).get()
print(maps)
print(f"Viewshed num cells: {gs.raster_info(maps[0])['cells']}")
print(f"DSM num cells: {gs.raster_info('dsm')['cells']}")


## Temporal dataset of viewsheds

In this part we will create, analyze and visualize a temporal dataset of viewsheds using [temporal tools](https://grass.osgeo.org/grass-stable/manuals/temporal.html). 

First, let's check we have the viewshed rasters ready:

In [ ]:
gs.list_strings(type="raster", pattern="viewshed_*")

We will create an empty space-time raster dataset called _viewsheds_ with relative temporal type:

In [ ]:
gs.run_command("t.create", output="viewsheds", type="strds", temporaltype="relative",
              title="Viewshed series", description="Series of viewsheds along a road")

Now we register the viewshed rasters with start time 1 and 1-minute increment to simulate a change of view of a car driving slowly along the road:

In [ ]:
gs.run_command("t.register", input="viewsheds", maps=",".join(maps), start=1, unit="minutes", increment=1)

Let's print basic dataset info. We will use this info later on to set computational region covering the entire dataset.

In [ ]:
from pprint import pprint
info = gs.parse_command("t.info", input="viewsheds", flags="g")
pprint(info)

To list the individual rasters, we will use t.rast.list. Notice there is no end time, because we don't use interval data.

In [ ]:
from io import StringIO

pd.read_csv(StringIO(gs.read_command("t.rast.list", input="viewsheds", separator="comma", columns="name,start_time,end_time")))

We can quickly get basic statistics such as the size of the viewsheds (see _sum_ column for the number of visible cells):

In [ ]:
df = pd.read_csv(StringIO(gs.read_command("t.rast.univar", input="viewsheds", separator="comma")))
df

Let's find and visualize largest and smallest viewshed:

In [ ]:
largest = df.iloc[df[['sum']].idxmax()["sum"]].id
smallest = df.iloc[df[['sum']].idxmin()["sum"]].id

gs.run_command("g.region", raster=[largest, smallest], save="zoom_region")
viewshed_map = gj.Map(saved_region="zoom_region")
viewshed_map.d_rast(map="ortho")
viewshed_map.d_rast(map=largest, values=1)
viewshed_map.d_rast(map=smallest, values=1)
viewshed_map.d_vect(map="umstead_drive", color="white")
viewshed_map.show()

Let's compute a temporal dataset where values of each viewshed will represent the registered start time.

Before we do that, let's set computational region to match the bounding box of the entire dataset, here we use previous t.info output:

In [ ]:
gs.run_command("g.region", n=info["north"], s=info["south"], e=info["east"], w=info["west"])

We use temporal raster algebra. Here we compute a new temporal dataset _viewsheds_start_ so that for example viewshed with start time 5 has value 5 for visible area and no data for invisible area.

In [ ]:
gs.run_command("t.rast.mapcalc", inputs="viewsheds", output="viewsheds_start", basename="viewshed_start",
               expression="if (viewsheds == 0, null(), start_time())")

Set color of the newly computed time series:

In [ ]:
gs.run_command("t.rast.colors", input="viewsheds_start", color="plasma")

With TimeSeriesMap, we can interactively visualize the time series:

In [ ]:
timemap = gj.TimeSeriesMap(width=800)
timemap.d_rast(map="ortho")
timemap.d_vect(map="umstead_drive", )
timemap.add_raster_series("viewsheds_start")
timemap.show()

We can export an animated GIF:

In [ ]:
from IPython.display import Image

Image(timemap.save("animation.gif", duration=300))

## Cumulative viewshed
We can compute the cumulative viewshed, which aggregates viewsheds from multiple viewpoints. In this way you can e.g., identify the most frequently visible areas from the road.


Since our viewshed rasters are binary (0 invisible, 1 visible), we will use r.series method *sum*. Then we replace zeros with no data using r.null and set a new color ramp:

In [ ]:
# cumulative viewshed
gs.run_command("r.series", input=maps, output="cumulative_viewshed", method="sum")
gs.run_command("r.null", map="cumulative_viewshed", setnull=0)
gs.run_command("r.colors", map="cumulative_viewshed", color="plasma")

Let's visualize the results:

In [ ]:
cumulative_map = gj.InteractiveMap()
cumulative_map.add_raster("cumulative_viewshed", opacity=0.8)
cumulative_map.add_vector("umstead_drive")
cumulative_map.add_layer_control(position="bottomright")
cumulative_map.show()

And create a 3D rendering with draped cumulative viewshed over the DSM:

In [ ]:
map3d = gj.Map3D()
map3d.render(elevation_map="dsm", resolution_fine=1, color_map="cumulative_viewshed",
           vline="umstead_drive", vline_width=3, vline_color="white", light_brightness=50,
           position=[0.4, 0.8], height=3000, perspective=10)
map3d.overlay.d_legend(raster="cumulative_viewshed", at=(0, 30, 1, 7), use=[1, 2, 3, 4, 5, 6], flags="fb")
map3d.show()

## Data reprojection
Next, we will analyze the cumulative viewshed to see how much greenery a driver would see on the way. To do that we compute NDVI:

We reproject R and NIR Landsat bands from NCSPM sample dataset we already have available. Tool r.proj respects the current region (extent and resolution), but you can set resolution to certain value, we use 28.5 m which is the original resolution.


In [ ]:
gs.run_command("g.region", raster="dsm")
for band in [30, 40]:
    gs.run_command("r.proj", location="nc_spm_08_grass7", mapset="PERMANENT", input=f"lsat7_2002_{band}", method="nearest", resolution=28.5)

Compute NDVI:

In [ ]:
gs.run_command("i.vi", viname="ndvi", red="lsat7_2002_30", nir="lsat7_2002_40", output="ndvi")

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="ndvi")
ndvi_map.d_legend(raster="ndvi")
ndvi_map.show()

## Mask
Now let's analyze what is the distribution of NDVI within the visible area. We will mask the data by the visible area:


In [ ]:
gs.run_command("r.mask", raster="cumulative_viewshed")
data = gs.parse_command("r.univar", map="ndvi", flags="g")
print(f"Average NDVI of visible cells: {float(data['mean']):.2f} ± {float(data['stddev']):.2f}")

In [ ]:
ndvi_map = gj.Map()
ndvi_map.d_rast(map="ndvi")
ndvi_map.d_legend(raster="ndvi", flags="d")
ndvi_map.show()

Let's see the histogram of visible NDVI using d.histogram:

In [ ]:
histo = gj.Map(width=800, height=400)
histo.d_histogram(map="ndvi", bgcolor="grey")
histo.show()

## Read as numpy array
It is also easy to use the results as a numpy array and then use other Python libraries to analyze the data:

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from grass.script import array as garray

In [ ]:
ndvi = garray.array(mapname="ndvi", null='nan')
ndvi

In [ ]:
sns.set_style('darkgrid')
sns.histplot(ndvi.ravel(), kde=True)

Finally, remove the mask:

In [ ]:
gs.run_command("r.mask", flags="r")